In [119]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity 
from tensorflow import keras
from tensorflow.keras.layers import Bidirectional,Dense,GlobalMaxPooling1D,LSTM
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [2]:
data = pd.read_csv("../../Data/clean_data.csv")

In [9]:
bert_preproccessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [99]:
def create_embedding(sentence):
    preroccessed_sentence = bert_preproccessor(sentence)
    bert_embedding = bert_model(preroccessed_sentence)
    return bert_embedding["pooled_output"]

balh = create_embedding(["hellow orow"])

In [116]:
data["embedding"] = data["Tweet Text"].map(lambda x : create_embedding([x]))

In [59]:
X_train, X_test, y_train, y_test = train_test_split(data["embedding"],data["Informativeness"])

In [117]:
type(X_train)

pandas.core.series.Series

In [121]:
# Defining the model

input = keras.Input(shape=(),dtype="string",name="input")

layer = Bidirectional(LSTM(128,return_sequences=True))(create_embedding(input))
layer = Bidirectional(LSTM(128,return_sequences=True))(layer)
layer = GlobalMaxPooling1D()(layer)
layer = Dense(512, activation="relu")(layer)
layer = Dense(1, activation="sigmoid")(layer)

model = keras.Model(input=[input],output=[layer])
# model = keras.Sequential()
# model.add(Bidirectional(CuDNNLSTM(128,return_sequences=True), input_shape=(1,768)))
# model.add(Bidirectional(CuDNNLSTM(128,return_sequences=True)))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(512, activation="relu"))
# model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.fit(X_train,y_train,epochs=5)
model.summary()

ValueError: Input 0 of layer bidirectional_27 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 768)